## Importando as bibliotecas necessárias

Nesse notebook as bibliotecas utilizadas serão:

* Tensorflow - Para criação e treinamento das redes neurais e outras funções de pré-processamento de dados

E outras bibliotecas nativas do Python - time e os

In [1]:
import tensorflow as tf

import os
import time

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
arquivo_path = "combined.txt"

In [3]:
text = open(arquivo_path, 'rb').read().decode(encoding='utf-8')
print(f'O texto contém: {len(text)} caracteres')

vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

O texto contém: 44418189 caracteres
141 caracteres únicos


## Pré-processando os dados

Para a nossa rede, cada caracter precisa ser convertido em um valor numérico para identificação, uma vez que as redes neurais só trabalham com números.

In [4]:
caracteres_para_ids = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids_para_caracteres = tf.keras.layers.StringLookup(
    vocabulary=caracteres_para_ids.get_vocabulary(), invert=True, mask_token=None)


2025-04-05 11:35:38.763028: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-04-05 11:35:38.763211: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-04-05 11:35:38.763224: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1743863738.763554   68631 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743863738.763604   68631 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Função para converter de volta os IDs em textos
def text_from_ids(ids):
  return tf.strings.reduce_join(caracteres_para_ids(ids), axis=-1)

## Dividindo o texto em sequencias de entrada

Em seguida, precisamos dividir o texto em sequencias menores de entrada para que a nossa rede possa receber no treinamento


In [6]:
texto_em_ids = caracteres_para_ids(tf.strings.unicode_split(text, 'UTF-8'))
dataset_de_ids = tf.data.Dataset.from_tensor_slices(texto_em_ids)

In [7]:
tamanho_da_sequencia_em_caracteres = 100
dados_por_dataset = len(text) // (tamanho_da_sequencia_em_caracteres + 1)

In [8]:
sequencias = dataset_de_ids.batch(tamanho_da_sequencia_em_caracteres+1, drop_remainder=True)

In [9]:
def divide_input(sequencia):
    input_text = sequencia[:-1]
    target_text = sequencia[1:]
    return input_text, target_text

In [10]:
dataset = sequencias.map(divide_input)

## Criando os lotes de treinamento
Utilizando funções do Tensorflow, vamos converter os dados em um formato de entrada válido para a nossa rede neural

In [11]:
TAMANHO_BATCH = 64

# Tamanho do buffer para embaralhar os exemplos do dataset
TAMANHO_BUFFER = 10000

dataset = (
    dataset
    .shuffle(TAMANHO_BUFFER)
    .batch(TAMANHO_BATCH, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))



## Construindo o modelo
Utilizando o `keras.model`, vamos criar uma rede neural recorrente com duas camadas LSTM (Long short-term memory).
No artigo original, o autor cria um modelo muito semelhante com uma única camada GRU (Gated Recurrent Unit), que são mais baratas computacionalmente porém possuem uma menor eficiencia devido a sua arquitetura com 2 portões (atualização e redefinição), ao contrário da LSTM que possui 3 portões (entrada, esquecimento e saída).

A arquitetura da rede é bem simples:
- Uma camada embedding com a dimensão de 256
- Duas camadas LSTM com 2048 neuronios
- Uma camada dense com o mesmo tamanho que a quantidade de caracteres, onde cada neurônio representa um caracter

O uso de duas camadas LSTM garante que o modelo aprenda padrões de maneira eficiente e com maior precisão

In [12]:
dimensao_de_embedding = 256
neuronios_lstm = 2048

In [14]:
class ModeloDeLinguagem(tf.keras.Model):
  def __init__(self, tamanho_do_vocabulario, dimensao_de_embedding, neuronios_lstm):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(tamanho_do_vocabulario, dimensao_de_embedding)
    self.lstm1 = tf.keras.layers.LSTM(neuronios_lstm,
                                      return_sequences=True,
                                      return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(neuronios_lstm,
                                      return_sequences=True,
                                      return_state=True)
    self.dense = tf.keras.layers.Dense(tamanho_do_vocabulario)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = self.embedding(inputs, training=training)
    batch_size = tf.shape(inputs)[0]

    if states is None:
        h0_1 = tf.zeros((batch_size, self.lstm1.units))
        c0_1 = tf.zeros((batch_size, self.lstm1.units))
        x, h1, c1 = self.lstm1(x, initial_state=[h0_1, c0_1], training=training)

        h0_2 = tf.zeros((batch_size, self.lstm2.units))
        c0_2 = tf.zeros((batch_size, self.lstm2.units))
        x, h2, c2 = self.lstm2(x, initial_state=[h0_2, c0_2], training=training)
    else:
        (h1, c1), (h2, c2) = states
        x, h1, c1 = self.lstm1(x, initial_state=[h1, c1], training=training)
        x, h2, c2 = self.lstm2(x, initial_state=[h2, c2], training=training)

    x = self.dense(x, training=training)

    if return_state:
        return x, [(h1, c1), (h2, c2)]
    else:
        return x

In [15]:
model = ModeloDeLinguagem(
    tamanho_do_vocabulario=len(caracteres_para_ids.get_vocabulary()),
    dimensao_de_embedding=dimensao_de_embedding,
    neuronios_lstm=neuronios_lstm)

## Treinando o modelo!!!!


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
# Salva os pesos do modelo em checkpoints do treinamento

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, ids_para_caracteres, caracteres_para_ids)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Ao verme'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)